In [1]:
from datasets import list_datasets
from requests_cache import CachedSession
import requests
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time
import warnings
from concurrent.futures import ThreadPoolExecutor
from huggingface_hub import hf_hub_download
from tqdm import tqdm
import os

<h3>Extraction des données des datasets d'HuggingFace</h3>


<h4>1) Extraction des métadonnées</h4>

In [ ]:
"""
Fonction pour récupérer les informations (métadonnées) d'un dataset
    name : nom du dataset
    session : session avec cache
"""
def fetch_dataset_info(name, session):
    url = f"https://huggingface.co/api/datasets/{name}"

    while True:
        response = session.get(url, params={"full": "True"})
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print("Retrying in 15 seconds...")
            time.sleep(15)
        else:
            return None

In [ ]:

"""
Fonction pour récupérer les informations (métadonnées) de plusieurs datasets en parallèle
    dataset_names : liste des noms des datasets
    session : session avec cache
    max_workers : nombre de threads
"""
def retrieve_dataset_info(dataset_names,session,max_workers=15):
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_dataset_info, name, session) for name in dataset_names]
        for future in tqdm(futures, total=len(dataset_names), desc="Progress"):
            result = future.result()
            results.append(result)
    return results

In [ ]:
# Récupération des noms de tous les datasets disponibles sur HuggingFace
dataset_names = list_datasets()
print("nb total de dataset disponible sur huggingface :", len(dataset_names))

In [ ]:
# Création d'une session avec cache
session = CachedSession()
# Récupération des informations (métadonnées) des datasets
datasets_info = retrieve_dataset_info(dataset_names, session)
print("nb de dataset ayant des métadonnées :", len(datasets_info))

In [ ]:
# Création d'un fichier json avec les données des datasets
with open("datasets_info.json", "w", encoding="utf8") as f:
    json.dump(datasets_info, f, ensure_ascii=False, indent=2)

<h4>2) Nettoyage des métadonnées</h4>


In [2]:
# Lecture du fichier json
with open("datasets_info.json", "r", encoding="utf8") as f:
    datasets_info = json.load(f)

# Nettoyage des données
datasets_info_clean = []
for dataset in tqdm(datasets_info):
    if dataset is None:
        continue
    else:
        # On enlève les données inutiles pour réduire la taille du fichier json
        if "cardData" in dataset and "configs" in dataset["cardData"]:
            dataset["cardData"].pop("configs")
        if "siblings" in dataset:
            dataset.pop("siblings")
        if "tags" in dataset:
            dataset.pop("tags")
        datasets_info_clean.append(dataset)

print("dataset_info Nettoyé :")
print(datasets_info[0])
print("Nombre de datasets nettoyés :", len(datasets_info_clean))

100%|██████████| 122908/122908 [00:09<00:00, 13306.92it/s]

dataset_info Nettoyé :
{'_id': '621ffdd236468d709f181d58', 'id': 'acronym_identification', 'sha': '15ef643450d589d5883e289ffadeb03563e80a9e', 'lastModified': '2024-01-09T11:39:57.000Z', 'private': False, 'gated': False, 'disabled': False, 'paperswithcode_id': 'acronym-identification', 'downloads': 706, 'likes': 18, 'cardData': {'annotations_creators': ['expert-generated'], 'language_creators': ['found'], 'language': ['en'], 'license': ['mit'], 'multilinguality': ['monolingual'], 'size_categories': ['10K<n<100K'], 'source_datasets': ['original'], 'task_categories': ['token-classification'], 'task_ids': [], 'paperswithcode_id': 'acronym-identification', 'pretty_name': 'Acronym Identification Dataset', 'tags': ['acronym-identification'], 'dataset_info': {'features': [{'name': 'id', 'dtype': 'string'}, {'name': 'tokens', 'sequence': 'string'}, {'name': 'labels', 'sequence': {'class_label': {'names': {'0': 'B-long', '1': 'B-short', '2': 'I-long', '3': 'I-short', '4': 'O'}}}}], 'splits': [{'

<h4>3) Extraction des dataset_cards</h4>

In [ ]:
"""
Fonction pour télécharger le fichier README.md(datasetCard) d'un dataset
    repo_id : nom du dataset
"""
def download_dataset_card(repo_id):
    try:
        data_card_path = hf_hub_download(repo_id=repo_id, filename="README.md", repo_type="dataset")
        with open(data_card_path, "r", encoding="utf8") as f:
            dataset_card = f.read()
        return dataset_card
    except Exception as e:
        print(f"Erreur téléchargement datasetCard {repo_id} :", str(e))
        return None

In [ ]:
"""
Fonction pour récupérer les dataset cards et les ajouter aux métadonnées des datasets (datasets_info)
    datasets_info : liste contenant les métadonnées des datasets
"""
def retrieve_dataset_cards(datasets_info):
    nb_datasets_card = 0

    # Liste pour stocker les futurs résultats
    futures = []

    # Utilisation de ThreadPoolExecutor pour télécharger en parallèle
    with ThreadPoolExecutor(max_workers=10) as executor:
        for dataset_info in datasets_info:
            if "cardData" in dataset_info:
                # Soumettre une tâche de téléchargement pour chaque dataset
                future = executor.submit(download_dataset_card, dataset_info["id"])
                futures.append((dataset_info, future))

        # Attendre que toutes les tâches soient terminées
        for dataset_info, future in tqdm(futures, total=len(futures), desc="Progress"):
            dataset_card = future.result()
            if dataset_card:
                # Ajouter le dataset_card aux métadonnées du dataset
                dataset_info["dataset_card"] = dataset_card
                nb_datasets_card += 1

    print(f"Nombre de dataset_card récupérés : {nb_datasets_card} / {len(datasets_info)}")

In [ ]:
# Désactivation des warnings
warnings.filterwarnings("ignore")
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# Récupération les dataset cards
retrieve_dataset_cards(datasets_info_clean)

In [8]:
# Création d'un fichier json avec les métadonnées des datasets + les dataset_card
with open("datasets_info+card.json", "w", encoding="utf8") as f:
    json.dump(datasets_info, f, ensure_ascii=False, indent=2)

<h3>Traitement des datasetCards</h3>

<h4>1) Extraction de la description des datasetCards</h4>


In [9]:
# Lecture du fichier json
with open("datasets_info+card.json", "r", encoding="utf8") as f:
    datasets_info = json.load(f)
print("Nombre de datasets :", len(datasets_info))

Nombre de datasets : 122908


In [ ]:
def get_description_from_card(model_card):
    description_keywords =\
    ["Description", "description", "Summary", "summary", "Detail", "detail", "Dataset", "dataset"]
    
    description = ""
    lines = model_card.split("\n")
    
    for i in range(len(lines)):
        if lines[i].startswith("#"):
            for description_keyword in description_keywords:
                if description_keyword in lines[i]:
                    # On récupère la description du dataset
                    i+=1
                    while i < len(lines) and not lines[i].startswith("#"):
                        description = description + lines[i]
                        i+=1
                    return description
    return None 

In [ ]:
# Parcourir le fichier json des datasets pour récupérer les dataset cards
with open("datasets.json", "r", encoding="utf8") as f:
    datasets = json.load(f)

descriptions = []
nb_datasets = len(datasets)
nb_descriptions_found = 0
# Récupération des descriptions des dataset cards
for dataset in tqdm(datasets):
    if "dataset_card" in dataset:
        dataset_card = dataset["dataset_card"]
        description = get_description_from_card(dataset_card)
        if description:
            descriptions.append({"description":description, "id":dataset["id"]})
            nb_descriptions_found += 1

print(nb_descriptions_found)
print(nb_datasets)
print(f"Pourcentage de dataset cards avec description: {(nb_descriptions_found/nb_datasets)*100:.2f}%")
print(descriptions[:20])